# Preprocessing, Feature Engineering, Model Selection - 1st Attempt

In [1]:
import json
import warnings
from datetime import datetime, timedelta

import holidays
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    average_precision_score,
    classification_report,
    roc_auc_score,
)
from sklearn.model_selection import (
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

After performing the initial EDA (which can be found in the EDA notebook), we will now perform the following steps:
1. Preprocess the data: data impuation, data transformation, data normalization
2. Feature addition: add lots of new features (interaction, encoding, binning...) 
3. Feature selection- Statistical validation tests: perform statistical tests to see if the new features are useful.
4. Feature selection- cross validation tests: perform cross validation to see if the new features are useful.
5. Model selection: try different models and see which one performs the best.
6. Model tuning: tune the best model to see if we can improve the performance.

## Loading & cleaning the data

In [2]:
# load and clean the data

filepath = "train_dataset_full.csv"
df = pd.read_csv(filepath)

# clean the data
def clean_data(df):
    # remove entirely empty rows and fully duplicate rows
    df = df.dropna(how="all").drop_duplicates()
    
    # ensure the DateTime column is in datetime format
    df["DateTime"] = pd.to_datetime(df["DateTime"])

    # ensure values that are supposed to be ints are indeed so (and not unnecessarily floats)
    int_columns = ['campaign_id', 'webpage_id', 'product_category_1',
                   'age_level', 'user_depth', 'city_development_index']
    for col in int_columns:
        df[col] = df[col].apply(lambda x: int(x) if pd.notnull(x) else x)
    
    return df


df = clean_data(df)

## Data imputation

In the EDA, the missing values that stood out were in:
1. "product_category_2" (79.16% missing): we will creates a missing indicator feature (which might get dropped in future steps if deemed irrelevant) but won't impute values, as that could introduce misleading data, as there's simply too much missing data. 
2. "city_development_index" (27.60%): we will creates a missing indicator feature (which might get dropped in future steps if deemed irrelevant) and impute with the mode.
3. "gender", "age_level", "user_depth", "user_group_id": we will first attempt to fill from other sessions of the same user, but we'll fallback to global defaults as needed.
4. Else: we will automatically impute missing values for features with missing rates below 1%, using the mode for categorical variables (including those with low cardinality), and medians for numerical variables.

In [3]:
def impute_demographics_by_user(df):
    """
    Imputes demographic features (gender, age_level, user_depth) using other 
    sessions from the same user when available, then falls back to global defaults.
    
    The EDA showed these features have about 4.7% missing values and are generally
    consistent within users, making this a reliable approach.
    """
    df = df.copy()
    
    # first, attempt to fill missing demographics using other sessions from same user
    demographics = ['gender', 'age_level', 'user_depth', 'user_group_id']
    
    for demo in demographics:
        # group by user_id and apply forward fill and backward fill
        df[demo] = df.groupby('user_id')[demo].transform(
            lambda x: x.ffill().bfill()
        )
    
    # if there are any remaining missing values, fill with mode values:
    defaults = {"gender": df["gender"].mode().iloc[0],
                "age_level": df["age_level"].mode().iloc[0],
                "user_depth": df["user_depth"].mode().iloc[0],
                "user_group_id": df["user_group_id"].mode().iloc[0]}

    
    for column, default in defaults.items():
        df[column] = df[column].fillna(default)
    
    return df


def handle_product_category2(df):
    """
    Handles product_category_2 which has 79.16% missing values.
    Creates a binary indicator for missingness and leaves original values as is.
    """
    df = df.copy()
    
    # create missing indicator
    df['product_category_2_missing'] = df['product_category_2'].isna().astype(int)
    
    # return without imputing due to extremely high missingness
    return df


def handle_city_development(df):
    """
    Handles city_development_index which has 27.60% missing values.
    Creates a missing indicator and imputes with mode.
    """
    df = df.copy()
    
    # create missing indicator
    df['city_development_missing'] = df['city_development_index'].isna().astype(int)
    
    # impute with mode
    city_development_mode = df['city_development_index'].mode()[0]
    df['city_development_index'] = df['city_development_index'].fillna(city_development_mode)
    
    return df


def handle_low_missing(df, threshold: float=0.01):
    """
    Handles features with low missing rates (< threshold).
    Uses mode for categorical variables and median for numerical ones.
    """
    df = df.copy()
    
    # identify columns with low missing rates
    missing_rates = df.isnull().mean()
    low_missing_cols = missing_rates[missing_rates > 0][missing_rates < threshold].index
    
    for col in low_missing_cols:
        # determine if column should be treated as categorical
        unique_vals = df[col].nunique()
        is_categorical = pd.api.types.is_object_dtype(df[col]) or unique_vals <= 10
        
        if is_categorical:
            # for categorical variables, use mode
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            # for numerical variables, use median
            df[col] = df[col].fillna(df[col].median())
    
    return df


def impute_dataset(df):
    """
    Applies the complete imputation pipeline to the dataset.
    Returns a new dataframe with all imputation strategies applied.
    """
    df = df.copy()
    
    # apply each imputation step in sequence
    df = impute_demographics_by_user(df)
    df = handle_product_category2(df)
    df = handle_city_development(df)
    df = handle_low_missing(df)
    
    return df



df_imputed = impute_dataset(df)

### validation of the imputation: looking at column distributions changes

In [4]:
def validate_imputation(df_original, df_imputed):
    """Validates that imputation maintained reasonable distributions"""
    for col in df_original.columns:
        if col == 'product_category_2':  # skip intentionally non-imputed column
            continue
            
        if pd.api.types.is_numeric_dtype(df_original[col]):
            # check that means and stds are similar for numeric columns
            orig_mean = df_original[col].mean()
            imp_mean = df_imputed[col].mean()
            orig_std = df_original[col].std()
            imp_std = df_imputed[col].std()
            
            print(f"\n{col}:")
            print(f"Mean - Original: {orig_mean:.2f}, Imputed: {imp_mean:.2f}")
            print(f"Std  - Original: {orig_std:.2f}, Imputed: {imp_std:.2f}")
        
        elif pd.api.types.is_object_dtype(df_original[col]):
            # check value distributions for categorical columns
            orig_dist = df_original[col].value_counts(normalize=True)
            imp_dist = df_imputed[col].value_counts(normalize=True)
            
            print(f"\n{col} distribution:")
            print("Original vs Imputed:")
            print(pd.concat([orig_dist, imp_dist], axis=1, 
                          keys=['Original', 'Imputed']).head())
            

imputation_params = {
    'demographics_defaults': {
        'gender': 'Male',
        'age_level': 3.0,
        'user_depth': 3.0,
        'user_group_id': 3.0
    },
    'city_development_mode': 2.0
}

# save for later use
with open('imputation_params.json', 'w') as f:
    json.dump(imputation_params, f)


def print_imputation_summary(df_original, df_imputed):
    """Prints summary of imputation changes"""
    total_missing_before = df_original.isnull().sum().sum()
    total_missing_after = df_imputed.isnull().sum().sum()
    
    print("\nImputation Summary:")
    print(f"Total missing values before: {total_missing_before:,}")
    print(f"Total missing values after: {total_missing_after:,}")
    print(f"Total values imputed: {total_missing_before - total_missing_after:,}")

validate_imputation(df, df_imputed)


session_id:
Mean - Original: 285452.08, Imputed: 285451.90
Std  - Original: 168593.35, Imputed: 167914.47

user_id:
Mean - Original: 545905.09, Imputed: 545790.67
Std  - Original: 329529.65, Imputed: 328225.35

product distribution:
Original vs Imputed:
         Original   Imputed
product                    
C        0.353391  0.358681
H        0.235999  0.234069
I        0.137374  0.136250
D        0.088477  0.087753
B        0.048436  0.048040

campaign_id:
Mean - Original: 308547.20, Imputed: 308968.33
Std  - Original: 126502.49, Imputed: 126063.30

webpage_id:
Mean - Original: 29699.44, Imputed: 29569.53
Std  - Original: 21548.50, Imputed: 21508.07

product_category_1:
Mean - Original: 3.07, Imputed: 3.08
Std  - Original: 1.30, Imputed: 1.30

user_group_id:
Mean - Original: 3.48, Imputed: 3.46
Std  - Original: 2.42, Imputed: 2.36

gender distribution:
Original vs Imputed:
        Original   Imputed
gender                    
Male    0.883568  0.889101
Female  0.116432  0.110899

a

Looking at the above validation, we can see that:
1. Our target feature "is_click" maintained exactly same means and std, meaning its integrity is preserved.
2. All features maintain very similar proportions apart from the below; "gender" ratio slightly shifted but reasonably (Male: 88.3% → 88.9%) but these small shifts are acceptable given the missingness rates.
3. "city_development_index" shows the largest shift, but it's expected given its high missingness rate (27.6%) and our mode imputation strategy.

## Feature Engineering: creating new features

In [5]:
def create_temporal_features(df):
    """
    Creates temporal features from DateTime column.
    """
    df = df.copy()
    
    # basic time features
    df['hour'] = df['DateTime'].dt.hour
    df['day_of_week'] = df['DateTime'].dt.dayofweek
    
    # binary features
    df['is_business_hours'] = df['hour'].between(9, 17).astype(int)
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['is_early_morning'] = df['hour'].between(2, 5).astype(int)
    
    # time of day as categorical with both raw and binned versions
    df['time_of_day'] = pd.cut(
        df['hour'],
        bins=[-np.inf, 6, 12, 18, np.inf],
        labels=['night', 'morning', 'afternoon', 'evening']
    )
    
    # add hour bins for reduced cardinality in interactions
    df['hour_bin'] = pd.cut(
        df['hour'],
        bins=[0, 4, 8, 12, 16, 20, 24],
        labels=['dawn', 'early_morning', 'morning', 'afternoon', 'evening', 'night'],
        include_lowest=True,  # include the lower bound
        right=False  # make intervals left-inclusive and right-exclusive
    )
    
    # holiday features
    us_holidays = holidays.US()
    df['date'] = df['DateTime'].dt.date
    
    # handle holidays with proper date handling
    df['is_holiday'] = df['date'].apply(
        lambda x: bool(us_holidays.get(x)) if pd.notna(x) else 0
    ).astype(int)
    
    # near holiday feature (2 days before/after)
    min_date = df['date'].min()
    max_date = df['date'].max()
    
    # filter holidays to our date range
    holiday_dates = [date for date in us_holidays.keys() 
                    if min_date <= date <= max_date]
    
    near_holiday_dates = set()
    for holiday in holiday_dates:
        near_holiday_dates.update([
            holiday - timedelta(days=i) for i in range(1, 3)
        ] + [
            holiday + timedelta(days=i) for i in range(1, 3)
        ])
    
    df['is_near_holiday'] = df['date'].apply(
        lambda x: x in near_holiday_dates if pd.notna(x) else 0
    ).astype(int)
    
    # clean up temporary column
    df = df.drop('date', axis=1)
    
    return df


def create_user_engagement_features(df):
    """
    Creates user engagement features with improved statistical handling.
    """
    df = df.copy()
    
    # sort by user and time for accurate historical features
    df = df.sort_values(['user_id', 'DateTime'])
    
    # historical CTR per user with smoothing
    user_clicks = df.groupby('user_id')['is_click'].agg(['sum', 'count']).reset_index()
    
    # add Bayesian smoothing
    global_ctr = df['is_click'].mean()
    smoothing_factor = 10  # Adjust based on data volume
    user_clicks['historical_user_ctr'] = (
        (user_clicks['sum'] + smoothing_factor * global_ctr) / 
        (user_clicks['count'] + smoothing_factor)
    )
    df = df.merge(user_clicks[['user_id', 'historical_user_ctr']], 
                 on='user_id', how='left')
    
    # session counts with log transformation
    df['session_count_user'] = df.groupby('user_id')['session_id'].transform('count')
    df['session_count_log'] = np.log1p(df['session_count_user'])
    
    # create session count bins using the log-transformed values
    df['session_count_bin'] = pd.qcut(
        df['session_count_log'],  # use the log-transformed feature
        q=5,  # quantile-based binning
        labels=['VL', 'L', 'M', 'H', 'VH']
    )
    
    # sessions per day with improved statistics
    df['date'] = df['DateTime'].dt.date
    sessions_per_day = df.groupby(['user_id', 'date']).size().reset_index()
    avg_sessions = sessions_per_day.groupby('user_id')[0].agg(['mean', 'std']).reset_index()
    avg_sessions.columns = ['user_id', 'sessions_per_day_mean', 'sessions_per_day_std']
    avg_sessions['sessions_per_day_std'] = avg_sessions['sessions_per_day_std'].fillna(0)
    df = df.merge(avg_sessions, on='user_id', how='left')
    
    # log transformation for sessions per day mean (optional, if skewed)
    df['sessions_per_day_mean_log'] = np.log1p(df['sessions_per_day_mean'])

    # time since last click
    max_hours = 168  # 1 week
    df['time_since_last_click'] = (
        df['DateTime'] - df.groupby('user_id')['DateTime'].shift(1)
    ).dt.total_seconds() / 3600
    df['time_since_last_click'] = df['time_since_last_click'].fillna(max_hours)
    
    # click frequency in last 24h with vectorized implementation
    df['prev_24h'] = df['DateTime'] - pd.Timedelta(hours=24)
    df['click_frequency_24h'] = df.groupby('user_id').apply(
        lambda group: group.apply(
            lambda row: group[
                (group['DateTime'] <= row['DateTime']) & 
                (group['DateTime'] >= row['prev_24h']) & 
                (group['is_click'] == 1)
            ].shape[0],
            axis=1
        )
    ).reset_index(level=0, drop=True)
    
    # log transform click frequency
    df['click_frequency_24h_log'] = np.log1p(df['click_frequency_24h'])
    
    # improved engagement score using percentile ranks
    engagement_features = [
        'historical_user_ctr',
        'session_count_log',  # log-transformed
        'sessions_per_day_mean_log',  # log-transformed mean
        'click_frequency_24h_log'  # log-transformed
    ]
    
    # convert each component to percentile rank
    for feature in engagement_features:
        df[f'{feature}_rank'] = df[feature].rank(pct=True)
    
    # weighted combination of ranks
    df['user_engagement_score'] = (
        0.4 * df['historical_user_ctr_rank'] +
        0.25 * df['session_count_log_rank'] +
        0.25 * df['sessions_per_day_mean_log_rank'] +
        0.1 * df['click_frequency_24h_log_rank']
        )
    
    # clean up temporary columns
    df = df.drop(['prev_24h', 'date'], axis=1)
    
    return df


def create_campaign_performance_features(df):
    """
    Creates campaign performance features with improved variance handling.
    """
    df = df.copy()
    
    # campaign historical CTR with log transformation
    campaign_stats = df.groupby('campaign_id')['is_click'].agg(['mean', 'count']).reset_index()
    campaign_stats.columns = ['campaign_id', 'campaign_historical_ctr', 'campaign_impressions']
    
    # log transform CTR after scaling to percentage
    campaign_stats['campaign_historical_ctr_log'] = np.log1p(campaign_stats['campaign_historical_ctr'] * 100)
    df = df.merge(campaign_stats, on='campaign_id', how='left')
    
    # create binned versions of CTR features
    unique_vals = df['campaign_historical_ctr'].dropna().unique()  # get unique quantiles
    n_bins = min(5, len(unique_vals))
    
    # campaign-webpage combination with relative performance
    campaign_webpage_stats = df.groupby(['campaign_id', 'webpage_id'])['is_click'].agg([
        ('campaign_webpage_ctr', 'mean'),
        ('webpage_impressions', 'count')
    ]).reset_index()

    
    # calculate relative performance
    campaign_avg = df.groupby('campaign_id')['is_click'].mean()
    campaign_webpage_stats['campaign_webpage_relative'] = (
        campaign_webpage_stats['campaign_webpage_ctr'] / 
        campaign_avg[campaign_webpage_stats['campaign_id']].values
    )
    
    df = df.merge(
        campaign_webpage_stats[['campaign_id', 'webpage_id', 'campaign_webpage_relative']], 
        on=['campaign_id', 'webpage_id'], 
        how='left'
    )
    
    # hourly performance with relative metrics
    hourly_stats = df.groupby(['hour'])['is_click'].mean().reset_index()
    hourly_stats.columns = ['hour', 'hour_ctr']
    df = df.merge(hourly_stats, on='hour', how='left')
    
    campaign_hour_stats = df.groupby(['campaign_id', 'hour'])['is_click'].mean().reset_index()
    campaign_hour_stats['campaign_hour_relative'] = (
        campaign_hour_stats['is_click'] / 
        campaign_hour_stats.groupby('campaign_id')['is_click'].transform('mean')
    )
    
    df = df.merge(
        campaign_hour_stats[['campaign_id', 'hour', 'campaign_hour_relative']], 
        on=['campaign_id', 'hour'], 
        how='left'
    )
    
    # product performance with relative metrics
    campaign_product_stats = df.groupby(['campaign_id', 'product_category_1'])['is_click'].agg([
        ('campaign_product_ctr', 'mean'),
        ('product_impressions', 'count')
    ]).reset_index()
    
    campaign_product_stats['campaign_product_relative'] = (
        campaign_product_stats['campaign_product_ctr'] / 
        campaign_avg[campaign_product_stats['campaign_id']].values
    )
    
    df = df.merge(
        campaign_product_stats[[
            'campaign_id', 'product_category_1', 'campaign_product_relative'
        ]], 
        on=['campaign_id', 'product_category_1'], 
        how='left'
    )
    
    # campaign success as percentile rank
    df['campaign_success_percentile'] = df['campaign_historical_ctr'].rank(pct=True)
    
    return df


def create_interaction_features(df):
    """
    Creates interaction features with improved cardinality handling.
    """
    df = df.copy()
    
    # time-based interactions using binned features
    df['campaign_hour_bin'] = df['campaign_id'].astype(str) + '_' + df['hour_bin'].astype(str)
    df['campaign_early_morning'] = df['campaign_id'] * df['is_early_morning']
    
    # user depth interactions with binned time
    df['user_depth_time'] = df['user_depth'].astype(str) + '_' + df['time_of_day'].astype(str)
    
    # age-weekend interaction (keep numeric for modeling)
    df['age_weekend'] = df['age_level'] * df['is_weekend']
    
    # user-based interactions
    df['user_depth_age'] = df['user_depth'] * df['age_level']
    
    # session-time interaction using binned sessions
    df['user_sessions_time'] = df['session_count_bin'].astype(str) + '_' + df['time_of_day'].astype(str)
    
    # demographic interactions
    df['gender_age'] = df['gender'].astype(str) + '_' + df['age_level'].astype(str)
    
    # campaign interactions using relative performance
    df['campaign_success_time'] = df['campaign_success_percentile'] * df['hour'].map(
        df.groupby('hour')['is_click'].mean()
    )
    
    # geographic interactions
    df['city_business'] = df['city_development_index'] * df['is_business_hours']
    
    return df


def create_all_features(df):
    """
    Applies all feature engineering steps and returns both engineered features
    and a list of features requiring scaling.
    """
    df = df.copy()
    
    # Apply each feature engineering step
    df = create_temporal_features(df)
    df = create_user_engagement_features(df)
    df = create_campaign_performance_features(df)
    df = create_interaction_features(df)
    
    # identify features that need scaling
    features_to_scale = [
        'historical_user_ctr',
        'session_count_log',
        'sessions_per_day_mean',
        'sessions_per_day_std',
        'time_since_last_click',
        'click_frequency_24h_log',
        'user_engagement_score',
        'campaign_historical_ctr_log',
        'campaign_webpage_relative',
        'campaign_hour_relative',
        'campaign_product_relative',
        'campaign_success_percentile',
        'user_depth_age',
        'age_weekend',
        'city_business',
        'campaign_success_time'
    ]
    
    return df, features_to_scale


df_engineered, features_to_scale = create_all_features(df_imputed)

### Validate the completness of the new features, etc.:

In [6]:
def validate_engineered_features(df):
    """
    Performs sanity checks on engineered features.
    """
    results = {}
    
    # check for nulls
    null_counts = df.isnull().sum()
    results['features_with_nulls'] = null_counts[null_counts > 0].to_dict()
    
    # check for infinite values
    inf_counts = df.isin([np.inf, -np.inf]).sum()
    results['features_with_inf'] = inf_counts[inf_counts > 0].to_dict()
    
    # validate binary features are actually binary
    binary_features = [col for col in df.columns if col.startswith('is_')]
    non_binary = {col: sorted(df[col].unique()) 
                 for col in binary_features 
                 if not df[col].isin([0, 1, np.nan]).all()}
    results['non_binary_features'] = non_binary
    
    # check for low variance features
    variances = df.select_dtypes(include=np.number).var()
    low_variance = variances[variances < 0.01].to_dict()
    results['low_variance_features'] = low_variance
    
    # validate time-based features
    if 'hour' in df.columns:
        results['hour_range_valid'] = df['hour'].between(0, 23).all()
    if 'day_of_week' in df.columns:
        results['day_range_valid'] = df['day_of_week'].between(0, 6).all()
        
    # check for correct CTR ranges (between 0 and 1)
    ctr_features = [col for col in df.columns if 'ctr' in col.lower()]
    invalid_ctr = {col: (df[col].min(), df[col].max()) 
                  for col in ctr_features 
                  if not df[col].between(0, 1, inclusive='both').all()}
    results['invalid_ctr_ranges'] = invalid_ctr
    
    # cardinality check for categorical features
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    high_cardinality = {col: df[col].nunique() 
                       for col in categorical_cols 
                       if df[col].nunique() > 100}
    results['high_cardinality_features'] = high_cardinality
    
    return results


def print_validation_results(results: dict):
    """
    Prints validation results in a readable format.
    """
    print("Feature Validation Results:")
    print("-" * 50)
    
    if results['features_with_nulls']:
        print("\nFeatures with null values:")
        for feat, count in results['features_with_nulls'].items():
            print(f"  {feat}: {count} nulls")
    
    if results['features_with_inf']:
        print("\nFeatures with infinite values:")
        for feat, count in results['features_with_inf'].items():
            print(f"  {feat}: {count} infinities")
    
    if results['non_binary_features']:
        print("\nNon-binary 'is_' features:")
        for feat, values in results['non_binary_features'].items():
            print(f"  {feat}: {values}")
    
    if results['low_variance_features']:
        print("\nFeatures with very low variance:")
        for feat, var in results['low_variance_features'].items():
            print(f"  {feat}: {var:.6f}")
    
    if results['invalid_ctr_ranges']:
        print("\nCTR features with invalid ranges:")
        for feat, (min_val, max_val) in results['invalid_ctr_ranges'].items():
            print(f"  {feat}: range [{min_val:.3f}, {max_val:.3f}]")
    
    if results['high_cardinality_features']:
        print("\nHigh cardinality categorical features:")
        for feat, count in results['high_cardinality_features'].items():
            print(f"  {feat}: {count} unique values")


validation_results = validate_engineered_features(df_engineered)
print_validation_results(validation_results)

Feature Validation Results:
--------------------------------------------------

Features with null values:
  product_category_2: 292615 nulls

Features with very low variance:
  historical_user_ctr: 0.001365
  campaign_historical_ctr: 0.000229
  campaign_webpage_relative: 0.001061
  hour_ctr: 0.000012
  campaign_success_time: 0.000370

CTR features with invalid ranges:
  campaign_historical_ctr_log: range [1.702, 2.314]


In [7]:
def prepare_features_for_modeling(df, categorical_features, features_to_scale, keep_as_is):
    """
    Prepares features for modeling by handling categorical and numerical variables.
    """
    df = df.copy()
    
    # verify all features are accounted for
    all_features = set(categorical_features + features_to_scale + keep_as_is)
    missing_features = set(df.columns) - all_features
    extra_features = all_features - set(df.columns)
    
    if missing_features:
        print(f"Warning: These columns are not categorized: {missing_features}")
    if extra_features:
        print(f"Warning: These categorized features are not in dataframe: {extra_features}")
    
    # ordinal encoding for ordinal features
    ordinal_features = {
        'age_level': range(7),
        'user_depth': range(1, 4),
        'city_development_index': range(1, 5)
    }
    
    for col, categories in ordinal_features.items():
        if col in df.columns:
            df[col] = pd.Categorical(df[col], categories=categories, ordered=True).codes
    
    # one-hot encoding for categorical features
    for col in categorical_features:
        if col in df.columns:
            dummies = pd.get_dummies(df[col], prefix=col, drop_first=True)
            df = pd.concat([df, dummies], axis=1)
            df = df.drop(col, axis=1)
    
    return df


categorical_features = [
    # Base categorical features
    'gender', 
    'product',
    
    # Temporal categoricals
    'time_of_day',    # night/morning/afternoon/evening
    'hour_bin',       # dawn/early_morning/morning/afternoon/evening/night
    
    # Binned features
    'session_count_bin',  # VL/L/M/H/VH
    
    # Interaction categoricals
    'campaign_hour_bin',  # campaign_id + hour_bin
    'user_depth_time',    # user_depth + time_of_day
    'user_sessions_time', # session_count_bin + time_of_day
    'gender_age'         # gender + age_level
]

features_to_scale = [
    # User engagement metrics
    'historical_user_ctr',
    'session_count_log',
    'sessions_per_day_mean',
    'sessions_per_day_std',
    'sessions_per_day_mean_log',
    'time_since_last_click',
    'click_frequency_24h_log',
    'user_engagement_score',
    
    # Ranking features
    'historical_user_ctr_rank',
    'session_count_log_rank',
    'sessions_per_day_mean_log_rank',
    'click_frequency_24h_log_rank',
    
    # Campaign performance metrics
    'campaign_historical_ctr',
    'campaign_historical_ctr_log',
    'campaign_webpage_relative',
    'campaign_hour_relative',
    'campaign_product_relative',
    'campaign_success_percentile',
    'hour_ctr',  # Added
    
    # Interaction numerics
    'user_depth_age',
    'age_weekend',
    'city_business',
    'campaign_success_time',
    'campaign_early_morning'
]

keep_as_is = [
    # IDs and timestamps
    'session_id',
    'user_id',
    'campaign_id',
    'webpage_id',
    'DateTime',  # Added
    
    # Target
    'is_click',
    
    # Binary features
    'is_business_hours',
    'is_weekend',
    'is_early_morning',
    'is_holiday',
    'is_near_holiday',
    'product_category_2_missing',
    'city_development_missing',
    'var_1',  # Added
    
    # Time features in fixed ranges
    'hour',         # 0-23
    'day_of_week',  # 0-6
    
    # Ordinal features
    'age_level',           # 0-6
    'user_depth',          # 1-3
    'city_development_index', # 1-4
    
    # Other features
    'product_category_1',  # 1-5
    'product_category_2',  # Added (kept as-is due to high missingness)
    'user_group_id',      # Added (derived from age and gender)
    'campaign_impressions'
]


# prepare the features for modeling by encoding categorical variables
df_prepared = prepare_features_for_modeling(
    df_engineered, 
    categorical_features, 
    features_to_scale, 
    keep_as_is
)

In [8]:
def prefilter_features(df: pd.DataFrame, target_col: str = 'is_click', verbose: bool = True):
    """
    Multi-stage feature pre-filtering process.
    Returns a list of features that pass all filtering stages.
    """
    df = df.copy()
    initial_features = len(df.columns) - 1  # exclude target
    if verbose:
        print(f"Starting with {initial_features} features")
    
    # remove constant and quasi-constant features
    def remove_low_variance_features(df, threshold=0.01):
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        variances = df[numeric_cols].var()
        low_var_features = variances[variances < threshold].index.tolist()
        
        if verbose:
            print(f"\nLow variance features removed ({len(low_var_features)}):")
            for f in low_var_features:
                print(f"- {f} (variance: {variances[f]:.6f})")
        
        return df.drop(columns=low_var_features)

    # remove highly correlated features
    def remove_correlated_features(df, threshold=0.95):
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        corr_matrix = df[numeric_cols].corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        
        high_corr_features = []
        seen_pairs = set()
        
        # find features to remove
        for col in upper.columns:
            # Get correlations above threshold
            high_corr = upper[col][upper[col] > threshold].index.tolist()
            for feat in high_corr:
                if (col, feat) not in seen_pairs and (feat, col) not in seen_pairs:
                    # Keep the one with higher correlation with target
                    corr_with_target = abs(df[[col, feat, target_col]].corr()[target_col])
                    if corr_with_target[col] < corr_with_target[feat]:
                        high_corr_features.append(col)
                    else:
                        high_corr_features.append(feat)
                    seen_pairs.add((col, feat))
        
        high_corr_features = list(set(high_corr_features))
        
        if verbose:
            print(f"\nHighly correlated features removed ({len(high_corr_features)}):")
            for f in high_corr_features:
                print(f"- {f}")
        
        return df.drop(columns=high_corr_features)

    # remove features with low mutual information
    def remove_low_mi_features(df, target_col, threshold=0.001):
        # separate numeric and categorical columns
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        numeric_cols.remove(target_col)
        
        # calculate MI scores for numeric features
        mi_scores = mutual_info_classif(
            StandardScaler().fit_transform(df[numeric_cols]), 
            df[target_col],
            random_state=42
        )
        mi_series = pd.Series(mi_scores, index=numeric_cols)
        
        # identify features with low MI scores
        low_mi_features = mi_series[mi_series < threshold].index.tolist()
        
        if verbose:
            print(f"\nLow mutual information features removed ({len(low_mi_features)}):")
            for f in low_mi_features:
                print(f"- {f} (MI score: {mi_series[f]:.6f})")
        
        return df.drop(columns=low_mi_features)

    # remove features with high p-value in univariate testing
    def remove_insignificant_features(df, target_col, p_threshold=0.05):
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        numeric_cols.remove(target_col)
        
        insignificant_features = []
        for col in numeric_cols:
            _, p_value = stats.mannwhitneyu(
                df[df[target_col] == 1][col],
                df[df[target_col] == 0][col],
                alternative='two-sided'
            )
            if p_value > p_threshold:
                insignificant_features.append(col)
        
        if verbose:
            print(f"\nStatistically insignificant features removed ({len(insignificant_features)}):")
            for f in insignificant_features:
                print(f"- {f}")
        
        return df.drop(columns=insignificant_features)


    # apply all filtering stages
    df = remove_low_variance_features(df)
    df = remove_correlated_features(df)
    df = remove_low_mi_features(df, target_col)
    df = remove_insignificant_features(df, target_col)
    
    selected_features = [col for col in df.columns if col != target_col]
    
    if verbose:
        print(f"\nFinal feature set: {len(selected_features)} features")
        print("\nRemaining features:")
        for f in selected_features:
            print(f"- {f}")
    
    return selected_features

In [9]:
# convert boolean columns to integers
bool_cols = df_prepared.select_dtypes(include='bool').columns
df_prepared[bool_cols] = df_prepared[bool_cols].astype(int)

# omit the "product_category_2" feature as it has high missingness and was replaced
# by a binary indicator
df_prepared = df_prepared.drop('product_category_2', axis=1)

# run through the pre-filtering process to see its suggestions
selected_features = prefilter_features(df_prepared)

Starting with 171 features

Low variance features removed (33):
- historical_user_ctr (variance: 0.001365)
- campaign_historical_ctr (variance: 0.000229)
- campaign_webpage_relative (variance: 0.001061)
- hour_ctr (variance: 0.000012)
- campaign_success_time (variance: 0.000370)
- campaign_hour_bin_105960.0_dawn (variance: 0.002765)
- campaign_hour_bin_105960.0_evening (variance: 0.008292)
- campaign_hour_bin_105960.0_night (variance: 0.005713)
- campaign_hour_bin_118601.0_dawn (variance: 0.002693)
- campaign_hour_bin_359520.0_dawn (variance: 0.005111)
- campaign_hour_bin_360936.0_dawn (variance: 0.003241)
- campaign_hour_bin_396664.0_afternoon (variance: 0.009901)
- campaign_hour_bin_396664.0_dawn (variance: 0.001898)
- campaign_hour_bin_396664.0_early_morning (variance: 0.007651)
- campaign_hour_bin_396664.0_morning (variance: 0.008215)
- campaign_hour_bin_404347.0_dawn (variance: 0.002617)
- campaign_hour_bin_404347.0_early_morning (variance: 0.009052)
- campaign_hour_bin_405490.0_d

In [10]:
# define columns to drop
base_cols_to_drop = ['is_click', 'DateTime', 'session_id', 'user_id', 
                     'sessions_per_day_std', 'historical_user_ctr', 'session_count_log_rank', 
                     'sessions_per_day_mean_log_rank', 'click_frequency_24h_log_rank',
                     'session_count_user', 'sessions_per_day_mean', 'click_frequency_24h', 
                     'campaign_historical_ctr']

# get pattern-matched columns
user_sessions_time_cols = [col for col in df_prepared.columns if 'user_sessions_time_' in col]
campaign_hour_bin_cols = [col for col in df_prepared.columns if 'campaign_hour_bin_' in col]
time_of_day_cols = [col for col in df_prepared.columns if 'time_of_day_' in col]

# combine all columns to drop
all_cols_to_drop = base_cols_to_drop + user_sessions_time_cols + campaign_hour_bin_cols + time_of_day_cols

# update the selected features list to exclude the columns to drop
selected_features = [col for col in selected_features if col not in all_cols_to_drop]

# remove these from the "categorical_features", "features_to_scale", and "keep_as_is" lists
categorical_features = [col for col in categorical_features if col not in all_cols_to_drop]
features_to_scale = [col for col in features_to_scale if col not in all_cols_to_drop]
keep_as_is = [col for col in keep_as_is if col not in all_cols_to_drop]

# safely drop columns and split data- if the columns are not present, it will not raise an error
y = df_prepared['is_click']
X = df_prepared.drop(all_cols_to_drop, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
# scale the features that need it, by training the scaler on the training set
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[features_to_scale] = scaler.fit_transform(X_train[features_to_scale])
X_test_scaled[features_to_scale] = scaler.transform(X_test[features_to_scale])

## Feature Selection

In [12]:
def evaluate_feature_importance_optimized(X, y, n_bootstraps=100, sample_size=None):
    if sample_size is None:
        sample_size = len(X)

    mi_scores = np.zeros((n_bootstraps, X.shape[1]), dtype=np.float32)
    rf_importance = np.zeros((n_bootstraps, X.shape[1]), dtype=np.float32)

    rf = RandomForestClassifier(
        n_estimators=30,
        max_depth=10,
        n_jobs=-1,
        random_state=42,
    )

    indices_cache = np.random.randint(0, len(X), size=(n_bootstraps, sample_size))
    for i, indices in enumerate(indices_cache):
        if i % 10 == 0:
            print(f"Bootstrap iteration {i}/{n_bootstraps}")

        X_boot, y_boot = X.iloc[indices], y.iloc[indices]

        mi_scores[i] = mutual_info_classif(X_boot, y_boot, random_state=42)
        rf.fit(X_boot, y_boot)
        rf_importance[i] = rf.feature_importances_

    mean_std_ci = lambda scores: (
        scores.mean(),
        scores.std(),
        stats.norm.interval(0.95, scores.mean(), scores.std())
    )

    results = []
    for j, feature in enumerate(X.columns):
        mi_mean, mi_std, mi_ci = mean_std_ci(mi_scores[:, j])
        rf_mean, rf_std, rf_ci = mean_std_ci(rf_importance[:, j])
        results.append({
            'feature': feature,
            'mi_score': mi_mean,
            'mi_ci_lower': mi_ci[0],
            'mi_ci_upper': mi_ci[1],
            'rf_importance': rf_mean,
            'rf_ci_lower': rf_ci[0],
            'rf_ci_upper': rf_ci[1],
        })

    return pd.DataFrame(results)


def select_features_cv_optimized(X, y, base_features, threshold=0.01):
    selected_features = base_features.copy()
    remaining_features = [f for f in X.columns if f not in selected_features]

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    estimator = RandomForestClassifier(
        n_estimators=30,
        max_depth=10,
        n_jobs=-1,
        random_state=42
    )

    base_score = cross_val_score(
        estimator,
        X[selected_features],
        y,
        cv=cv,
        scoring='f1',
    ).mean()

    improved = True
    while improved and remaining_features:
        improved = False
        scores = {}

        for feature in remaining_features:
            current_features = selected_features + [feature]
            score = cross_val_score(
                estimator,
                X[current_features],
                y,
                cv=cv,
                scoring='f1',
                n_jobs=-1,
            ).mean()
            scores[feature] = score

        best_feature, best_score = max(scores.items(), key=lambda x: x[1])
        if best_score > base_score + threshold:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            base_score = best_score
            print(f"Added {best_feature} (new f1: {best_score:.4f})")
            improved = True

    return selected_features


def select_features_optimized(X_train_scaled, y_train, X_test_scaled=None):

    importance_df = evaluate_feature_importance_optimized(X_train_scaled, y_train)

    stable_features = importance_df[
        (importance_df['mi_ci_lower'] > 0) &
        (importance_df['rf_ci_lower'] > 0)
    ]['feature'].tolist()

    print(f"\nFound {len(stable_features)} stable features")

    base_features = importance_df[
        (importance_df['mi_score'] > importance_df['mi_score'].median()) &
        (importance_df['rf_importance'] > importance_df['rf_importance'].median())
    ]['feature'].tolist()

    selected_features = select_features_cv_optimized(
        X_train_scaled[stable_features],
        y_train,
        base_features,
    )

    print("\nFeature Selection Summary:")
    print(f"Initial features: {X_train_scaled.shape[1]}")
    print(f"Stable features: {len(stable_features)}")
    print(f"Final selected features: {len(selected_features)}")

    return selected_features, importance_df

In [13]:
# call the optimized feature selection function
selected_features, importance_df = select_features_optimized(X_train_scaled, y_train, X_test_scaled)

# review results
print("\nSelected Features:")
print(selected_features)

print("\nFeature Importance DataFrame:")
print(importance_df.sort_values(by='rf_importance', ascending=False).head())

Bootstrap iteration 0/100
Bootstrap iteration 10/100
Bootstrap iteration 20/100
Bootstrap iteration 30/100
Bootstrap iteration 40/100
Bootstrap iteration 50/100
Bootstrap iteration 60/100
Bootstrap iteration 70/100
Bootstrap iteration 80/100
Bootstrap iteration 90/100

Found 58 stable features

Feature Selection Summary:
Initial features: 78
Stable features: 58
Final selected features: 28

Selected Features:
['campaign_id', 'webpage_id', 'product_category_1', 'user_group_id', 'age_level', 'city_development_index', 'var_1', 'city_development_missing', 'day_of_week', 'is_near_holiday', 'session_count_log', 'sessions_per_day_mean_log', 'time_since_last_click', 'click_frequency_24h_log', 'historical_user_ctr_rank', 'user_engagement_score', 'campaign_impressions', 'campaign_historical_ctr_log', 'campaign_success_percentile', 'campaign_early_morning', 'age_weekend', 'user_depth_age', 'city_business', 'product_C', 'product_H', 'session_count_bin_H', 'session_count_bin_VH', 'gender_age_Male_3.

In [21]:
final_features = ['campaign_id', 'webpage_id', 'product_category_1', 'user_group_id', 'age_level', 
                 'city_development_index', 'var_1', 'city_development_missing', 'day_of_week', 
                 'is_near_holiday', 'session_count_log', 'sessions_per_day_mean_log', 
                 'time_since_last_click', 'click_frequency_24h_log', 'historical_user_ctr_rank', 
                 'user_engagement_score', 'campaign_impressions', 'campaign_historical_ctr_log', 
                 'campaign_success_percentile', 'campaign_early_morning', 'age_weekend', 
                 'user_depth_age', 'city_business', 'product_C', 'product_H', 'session_count_bin_H', 
                 'session_count_bin_VH', 'gender_age_Male_3.0']

In [23]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    
    # calculate metrics
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    avg_precision = average_precision_score(y_test, y_pred_proba)
    
    # cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    
    print(f"\n{model_name} Results:")
    print("-" * 50)
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"Average Precision Score: {avg_precision:.4f}")
    print(f"Cross-validation ROC AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return model


# keep only selected features in train and test sets
X_train_final = X_train_scaled[selected_features]
X_test_final = X_test_scaled[selected_features]


# logistic regression
lr_model = evaluate_model(
    LogisticRegression(max_iter=1000, class_weight='balanced'),
    X_train_final, X_test_final, y_train, y_test,
    "Logistic Regression"
)

# random forest
rf_model = evaluate_model(
    RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        class_weight='balanced',
        n_jobs=-1,
        random_state=42
    ),
    X_train_final, X_test_final, y_train, y_test,
    "Random Forest"
)

# gradient boosting
gb_model = evaluate_model(
    GradientBoostingClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    ),
    X_train_final, X_test_final, y_train, y_test,
    "Gradient Boosting"
)


Logistic Regression Results:
--------------------------------------------------
ROC AUC Score: 0.9711
Average Precision Score: 0.5495
Cross-validation ROC AUC: 0.9702 (+/- 0.0016)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96     68969
         1.0       0.49      0.97      0.65      4962

    accuracy                           0.93     73931
   macro avg       0.74      0.95      0.80     73931
weighted avg       0.96      0.93      0.94     73931


Random Forest Results:
--------------------------------------------------
ROC AUC Score: 0.9875
Average Precision Score: 0.8669
Cross-validation ROC AUC: 0.9866 (+/- 0.0007)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.92      0.96     68969
         1.0       0.47      0.99      0.63      4962

    accuracy                           0.92     73931
   macro avg       0.73      0.95      0.80     739

In [ ]:
dtrain = xgb.DMatrix(X_train_final, label=y_train)
dtest = xgb.DMatrix(X_test_final, label=y_test)

params = {
    'max_depth': 5,
    'learning_rate': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': ['logloss', 'auc'],
    'seed': 42,
    # Add only minimal adjustments
    'min_child_weight': 3,  # Helps prevent overfitting
    'subsample': 0.9,      # Slight randomness
    'colsample_bytree': 0.9  # Slight feature sampling
}


# xgboost model
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=200,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=20,
    verbose_eval=50
)

# cross-validation
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=1000,
    nfold=5,
    early_stopping_rounds=20,
    metrics=['auc', 'logloss'],
    seed=42
)

In [32]:
y_pred_proba = xgb_model.predict(dtest)  # predict probabilities

# find the optimal threshold for F1 score
thresholds = np.arange(0.2, 0.8, 0.05)
f1_scores = []
for threshold in thresholds:
    y_pred_thresh = (y_pred_proba > threshold).astype(int)
    f1_scores.append(f1_score(y_test, y_pred_thresh))

optimal_threshold = thresholds[np.argmax(f1_scores)]
y_pred_optimal = (y_pred_proba > optimal_threshold).astype(int)

y_pred = (y_pred_proba > optimal_threshold).astype(int)

# feature importance
importance_dict = xgb_model.get_score(importance_type='gain')
importance_df = pd.DataFrame(
    [(k, v) for k, v in importance_dict.items()],
    columns=['feature', 'importance']
).sort_values('importance', ascending=False)

# print evaluation metrics
from sklearn.metrics import classification_report, f1_score, roc_auc_score
print("\nXGBoost Results:")
print("-" * 50)
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_proba))
print("F1 Score:", f1_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nTop 10 Most Important Features:")
print(importance_df.head(10))


XGBoost Results:
--------------------------------------------------
ROC AUC Score: 0.9885967890322466
F1 Score: 0.7649889713254462

Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     68969
         1.0       0.76      0.77      0.76      4962

    accuracy                           0.97     73931
   macro avg       0.87      0.88      0.87     73931
weighted avg       0.97      0.97      0.97     73931


Top 10 Most Important Features:
                        feature   importance
13      click_frequency_24h_log  2157.626953
12        time_since_last_click   463.921417
14     historical_user_ctr_rank   221.593262
11    sessions_per_day_mean_log   130.092178
15        user_engagement_score   104.153023
10            session_count_log    38.393528
18  campaign_success_percentile    20.707743
24                    product_H     9.741928
17  campaign_historical_ctr_log     9.685002
23                    product_C   